In [1]:
# specify the aws profile and region you want to use to deploy the stack
aws_profile="bbdev"
region_name="us-east-2"

MEGA = 1024*1024

In [2]:
lambda_name = !docker run \
    --rm \
    -v $HOME/.aws/credentials:/creds:ro \
    -v cloud-reader-tf:/infra/ \
    cloudfuse/cloud-reader-terraform output lambda_arn
lambda_name = lambda_name[0][1:len(lambda_name[0])-1]
print('lambda_name:', lambda_name)

lambda_name: arn:aws:lambda:us-east-2:615900053518:function:cloud-reader-benchmark


In [21]:
import boto3
import json
import base64

session = boto3.Session(profile_name=aws_profile)
client = session.client('lambda', region_name = region_name)

In [22]:
def bench_setup_1(ranges):
    inputParams = {
        "region": region_name,
        "bucket": "cloudfuse-taxi-data",
        "key": "tpch/tbl-s1/lineitem.tbl",
        "size": 759863287,
        "ranges": ranges
    }
    response = client.invoke(
        FunctionName = lambda_name,
        InvocationType = 'RequestResponse',
        Payload = json.dumps(inputParams),
        LogType='Tail'
    )
    print(base64.b64decode(response['LogResult']).decode("utf-8") )
    print(json.load(response['Payload']))


In [23]:
bench_setup_1([{"start": 0, "length": 100*MEGA}])

START RequestId: 72649036-e031-40aa-9acd-84909753a314 Version: $LATEST
END RequestId: 72649036-e031-40aa-9acd-84909753a314
REPORT RequestId: 72649036-e031-40aa-9acd-84909753a314	Duration: 1154.73 ms	Billed Duration: 1188 ms	Memory Size: 3008 MB	Max Memory Used: 132 MB	Init Duration: 32.32 ms	

{'init_duration': 19, 'range_durations': [1130]}


In [24]:
bench_setup_1([{"start": 0, "length": 100*MEGA}])

START RequestId: a3f43eed-35bd-4e1e-b448-cff4bf383c8d Version: $LATEST
END RequestId: a3f43eed-35bd-4e1e-b448-cff4bf383c8d
REPORT RequestId: a3f43eed-35bd-4e1e-b448-cff4bf383c8d	Duration: 1053.79 ms	Billed Duration: 1054 ms	Memory Size: 3008 MB	Max Memory Used: 232 MB	

{'init_duration': 0, 'range_durations': [1052]}


In [25]:
bench_setup_1([{"start": 0, "length": 100*MEGA}])

START RequestId: 02d89d2c-1d8d-48f6-966e-02c776606906 Version: $LATEST
END RequestId: 02d89d2c-1d8d-48f6-966e-02c776606906
REPORT RequestId: 02d89d2c-1d8d-48f6-966e-02c776606906	Duration: 1062.73 ms	Billed Duration: 1063 ms	Memory Size: 3008 MB	Max Memory Used: 333 MB	

{'init_duration': 0, 'range_durations': [1061]}


In [28]:
bench_setup_1([
    {"start": 0, "length": 100*MEGA},
    {"start": 100*MEGA, "length": 100*MEGA},
    {"start": 200*MEGA, "length": 100*MEGA},
])

START RequestId: e6e65dee-8668-4cd1-873e-4585939ea0fd Version: $LATEST
END RequestId: e6e65dee-8668-4cd1-873e-4585939ea0fd
REPORT RequestId: e6e65dee-8668-4cd1-873e-4585939ea0fd	Duration: 2281.23 ms	Billed Duration: 2282 ms	Memory Size: 3008 MB	Max Memory Used: 1436 MB	

{'init_duration': 0, 'range_durations': [2280, 2280, 2280]}


In [27]:
bench_setup_1([
    {"start": 0, "length": 100*MEGA},
    {"start": 100*MEGA, "length": 100*MEGA},
    {"start": 200*MEGA, "length": 100*MEGA},
    {"start": 300*MEGA, "length": 100*MEGA},
    {"start": 400*MEGA, "length": 100*MEGA},
    {"start": 500*MEGA, "length": 100*MEGA},
])

START RequestId: b41c1911-520f-4d33-9b90-6cceb83124af Version: $LATEST
END RequestId: b41c1911-520f-4d33-9b90-6cceb83124af
REPORT RequestId: b41c1911-520f-4d33-9b90-6cceb83124af	Duration: 6289.34 ms	Billed Duration: 6290 ms	Memory Size: 3008 MB	Max Memory Used: 1137 MB	

{'init_duration': 0, 'range_durations': [3498, 3498, 6288, 6288, 6288, 6288]}
